In [ ]:
# Common Stuffs
import os, sys, time, datetime, collections

In [ ]:
# Tensorflow Stuffs
import tensorflow as tf
import pandas as pd
from collections import deque
import numpy as np
import random

In [ ]:
from tqdm import tqdm_notebook, tqdm
from tqdm import tnrange as trange

In [ ]:
##/////////| OPTIONS |\\\\\\\\\\\\\\\# 
num_gpus = 4
replay_buffer = deque()
time_step = 0
epsilon = .09
gamma = .09
replay_size = 500
state_dim = 40
action_dim = 3
stats.g_step = 0
batch_size = 32

In [ ]:
# get dataset.
# dataset = np.load('testset.npy')
# dataset = np.load('../Megaset.alphagriffin.npy')
input_shape = dataset.shape
print(input_shape)

In [ ]:
print(len(dataset[0]), dataset[0])

In [ ]:
state_input = tf.placeholder('float', [None, 60])

In [ ]:
global_step = tf.Variable( 0, name='global_step', trainable=False )
learning_rate = tf.train.exponential_decay( .095,
                                            global_step,
                                            .000005,
                                            0.87,
                                            staircase=True,
                                            name="Learn_decay" )

In [ ]:
sigma = 1
weight_initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)
# weight_initializer = tf.variance_scaling_initializer(mode="fan_avg", distribution="uniform", scale=sigma)
bias_initializer = tf.zeros_initializer()

In [ ]:
def weight_variable(shape, stddev=0.1):
    initial = tf.truncated_normal(shape, stddev=stddev)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
c = []
for i in trange(num_gpus):
    with tf.device('/gpu:{}'.format(i)):
        W0 = weight_variable([120, 1024])
        B0 = bias_variable([1024])
        hidden_1 = tf.nn.relu(tf.add(tf.matmul(state_input, W0), B0))
        c.append(hidden_1)
Q_value = tf.add_n(c)

In [ ]:
# Output variables
action_input = tf.placeholder('float', [None, 3])
y_input = tf.placeholder('float', [None])

# Training Method
Q_action = tf.reduce_sum(tf.multiply(Q_value, action_input), reduction_indices=1)
cost = tf.reduce_mean(tf.square(y_input - Q_action))
optimizer = tf.train.AdamOptimizer(lr).minimize(
    cost, global_step=global_step, colocate_gradients_with_ops=True
)

In [1]:
def egreedy_action(state):
    Q_value = self.Q_value.eval(feed_dict={
        state_input: [state]
    })
    Q_value = Q_value[0]

    if epsilon <= 0.1:
        epsilon_rate = 1
    else:
        epsilon_rate = 0.95
    if time_step > 200:
        self.epsilon = epsilon_rate * epsilon

    if random.random() <= epsilon:
        return random.randint(0, 2)
    else:
        return np.argmax(Q_value)

In [ ]:
def action(state):
    Q_value = Q_value(feed_dict={
        state_input: [state]
    })
    Q_value = Q_value[0]
    return np.argmax(Q_value)

In [ ]:
def train( state, action, reward, state_, done):
        time_step += 1
        one_hot_action = np.zeros(3)
        one_hot_action[action] = 1
        replay_buffer.append((state, one_hot_action, reward, state_, done))
        if len(replay_buffer) > replay_size:  # this is a rolling window
            replay_buffer.popleft()
        if len(replay_buffer) > 100:  # after 100 step ,pre  train
            training()

In [ ]:
def training(self):
        # get random  sample from replay buffer
        minibatch = random.sample(replay_buffer, batch_size)
        state_batch = [data[0] for data in minibatch]
        action_batch = [data[1] for data in minibatch]
        reward_batch = [data[2] for data in minibatch]
        next_state_batch = [data[3] for data in minibatch]

        # calcuate Q
        Y_batch = []
        next_Q = self.Q_value.eval(feed_dict={
            state_input: next_state_batch
        })

        # Build a batch
        for i in range(batch_size):
            done = minibatch[i][4]
        if done:
            Y_batch.append(reward_batch[i])
        else:
            Y_batch.append(reward_batch[i] + gamma * np.max(next_Q[i]))

        # Train on that built Batch. Boom.
        """ EXAMPLE
        self.optimizer.run(feed_dict ={
        self.y_input:Y_bach,
        self.action_input:action_bach,
        self.state_input:state_bach
        })
            """
        FEED = {
            y_input: Y_batch,
            action_input: action_batch,
            state_input: state_batch
        }


        # //////| Training Method |\\\\\\\#
        _, stats.cost, stats.g_step = session.run([
                optimizer,
                cost,
                global_step],
                feed_dict=FEED
        )

In [ ]:
data = ss
session = tf.InteractiveSession(config=tf.ConfigProto(
                allow_soft_placement=True,
                ))
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()

In [ ]:
for i in trange(5):
    for j in range(5):
        iter_data = data[
            iter_step * 240: iter_step * 240 + 240
            ]
        env = StockEnv(iter_data)
        s = env.reset()
        while True:
            # first S is data 0 from first dataloop
            # s is recycled from bottom of loop
            action = Agent.egreedy_action(s)
            s_, reward, done = env.gostep(action)
            # print ("Action: {} | 0 = buy, 1 = sell".format(action))
            # print ("Reward: {:.4f}".format(reward))
            # print ("Current Price set: {}".format(s_))
            Agent.train(s, action, reward, s_, done)
            # print ("Prediction made...")
            # print("Total Cash on hand?: {}".format(env.cash))
            s = s_
            if done:
                # print("done")
                tqdm.write("Total Cash on hand?: {:.8f}".format(env.cash))
                tqdm.write("Total Global Steps: {}".format(Agent.stats.g_step))
                tqdm.write("Total Loss of Model: {}".format(Agent.stats.cost))
                break
    Agent.save_or_load()
    tqdm.write('Saved @ Global Step: {}, Total Loss: {}'.format(
        Agent.stats.g_step, Agent.stats.cost
    ))